In [2]:
from pdf2image import convert_from_path

pdf_path = 'metro.pdf'
dpi = 600

images = convert_from_path(pdf_path, dpi=dpi)

images[0].save(f'metro.png', 'PNG')

In [123]:
from PIL import Image
import numpy as np

image = Image.open("shape.png")

image_data = np.array(image)

black_color = (0, 0, 0, 255)

black_pixels = np.all(image_data[:, :, :3] == black_color[:3], axis=-1)

image_data[black_pixels, 3] = 0

result_image = Image.fromarray(image_data)

result_image.save("shape_transparent.png")

In [1]:
from PIL import Image
import os
import numpy as np

binary_mask = Image.open('shape.png')
binary_mask = binary_mask.convert('1') 

image_folder = 'icons/'

output_folder = 'processed/'
os.makedirs(output_folder, exist_ok=True)

png_images = [f for f in os.listdir(image_folder) if f.endswith('.png')]

binary_mask = np.array(binary_mask)

for image_filename in png_images:
    image_path = os.path.join(image_folder, image_filename)
    image = Image.open(image_path)

    image_array = np.array(image)

    result_image = np.dstack([image_array[:, :, 0] * binary_mask, image_array[:, :, 1] * binary_mask, image_array[:, :, 2] * binary_mask])

    result_image = Image.fromarray(result_image)

    gray_image = result_image.convert('L')
    threshold = 200

    binary_image = gray_image.point(lambda p: p > threshold and 255)

    result_filename = os.path.join(output_folder, image_filename)
    binary_image.save(result_filename)

print("[ All images have been processed ]")

[ All images have been processed ]


In [7]:
import cv2
import os
import numpy as np

input_folder = 'icons'
output_folder = 'augmented_images_color'

# input_folder = 'processed'
# output_folder = 'augmented_images_B&W'

os.makedirs(output_folder, exist_ok=True)

def rotate(random, image):
	# Rotate by a random angle
	angle = random.uniform(-10, 10)
	image = cv2.warpAffine(image, cv2.getRotationMatrix2D((133 / 2, 133 / 2), angle, 1), (133, 133))

	return image

def brightnessAndContrast(random, image):
	# Randomly adjust brightness and contrast
	alpha = 1 + random.uniform(-0.2, 0.2)
	beta = 0 + random.uniform(-20, 20)
	image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

	return image

def GaussianNoise(image):
	# Randomly add Gaussian noise
	noise = np.random.normal(0, 20, image.shape).astype(np.uint8)
	image = cv2.add(image, noise)

	return image

def augmentImage(image):
	# Apply a random combination of augmentations
	random_seed = np.random.randint(1, 100)
	random = np.random.RandomState(random_seed)

	image = rotate(random, image)
	image = brightnessAndContrast(random, image)
	# image = GaussianNoise(image)
	
	return image

id = 0
for filename in os.listdir(input_folder):

	if filename.endswith('.jpg') or filename.endswith('.png'):

		image_path = os.path.join(input_folder, filename)
		image = cv2.imread(image_path)
		for circle in range(0, 100):
			augmented_image = augmentImage(image)
			output_path = os.path.join(output_folder, str(id) + '_' + filename)
			cv2.imwrite(output_path, augmented_image)
			id += 1

print("[ Data augmentation complete ]")

[ Data augmentation complete ]


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# folder_path = 'augmented_images_color'
folder_path = 'icons'

file_list = os.listdir(folder_path)

feature_vectors = []

for image_file in file_list:
	image_path = os.path.join(folder_path, image_file)
	print(image_path)

	with tf.device('/GPU:0'):
		img = image.load_img(image_path, target_size=(224, 224))
		img = image.img_to_array(img)
		img = preprocess_input(img)
		img = np.expand_dims(img, axis=0)

		# Extract features
		features = base_model.predict(img)

		# Apply global average pooling
		features = tf.keras.layers.GlobalAveragePooling2D()(features)

		# Flatten and normalize if necessary
		feature_vector = features.numpy().flatten()
		feature_vectors.append(feature_vector)

import pandas as pd

df = pd.DataFrame()

feature_vectors = np.transpose(feature_vectors)

for i, ndarray in enumerate(feature_vectors):
	column_name = f"{i + 1}"
	df[column_name] = ndarray.flatten()

df['class'] = 0
df['class'] = (df.index // 100)

df.to_csv('FeatureVectors.csv', index=False)

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import pandas as pd

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# folder_path = 'icons'
folder_path = 'augmented_images_color'
file_list = os.listdir(folder_path)

feature_vectors = []
percent = 0
for image_file in file_list:
    image_path = os.path.join(folder_path, image_file)
    print(image_path)
    print("[ {:.2f} % ]".format(round((percent/19500)*100, 2)))

    with tf.device('/GPU:0'):
        img = image.load_img(image_path, target_size=(224, 224))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        img = np.expand_dims(img, axis=0)

        features = base_model.predict(img)

        features = tf.keras.layers.GlobalAveragePooling2D()(features)

        feature_vector = features.numpy().flatten()
        feature_vectors.append(feature_vector)

    percent += 1
    

df = pd.DataFrame(feature_vectors)
df['class'] = df.index // 100

df.to_csv('FeatureVectors.csv', index=False)

print("[ Feature vectors obtained ]")

augmented_images_color\0_1.1.png
[ 0.00 % ]
1/1 [==============================] - 4s 4s/step
augmented_images_color\10000_3.14.png
[ 0.01 % ]
1/1 [==============================] - 0s 17ms/step
augmented_images_color\10001_3.14.png
[ 0.01 % ]
1/1 [==============================] - 0s 16ms/step
augmented_images_color\10002_3.14.png
[ 0.02 % ]
1/1 [==============================] - 0s 16ms/step
augmented_images_color\10003_3.14.png
[ 0.02 % ]
1/1 [==============================] - 0s 19ms/step
augmented_images_color\10004_3.14.png
[ 0.03 % ]
1/1 [==============================] - 0s 17ms/step
augmented_images_color\10005_3.14.png
[ 0.03 % ]
1/1 [==============================] - 0s 19ms/step
augmented_images_color\10006_3.14.png
[ 0.04 % ]
1/1 [==============================] - 0s 15ms/step
augmented_images_color\10007_3.14.png
[ 0.04 % ]
1/1 [==============================] - 0s 17ms/step
augmented_images_color\10008_3.14.png
[ 0.05 % ]
1/1 [==============================] - 0s 17ms/st

In [6]:
lineas = []

for i in range(0,2000):
	lineas.append(1)

for i in range(0,2400):
	lineas.append(2)

for i in range(0,2100):
	lineas.append(3)

for i in range(0,1000):
	lineas.append(4)

for i in range(0,1300):
	lineas.append(5)
	
for i in range(0,1100):
	lineas.append(6)

for i in range(0,1400):
	lineas.append(7)

for i in range(0,1900):
	lineas.append(8)

for i in range(0,1200):
	lineas.append(9)

for i in range(0,1000):
	lineas.append(10)

for i in range(0,2100):
	lineas.append(11)

for i in range(0,2000):
	lineas.append(12)

print(len(lineas))

19500


In [7]:
import pandas as pd

df = pd.read_csv('FeatureVectors.csv')

df['lines'] = lineas

df.to_csv('FeatureVectors.csv', index=False)